In [1]:
from llm.llm import AWSBedrockLLm

sonnet35 = "anthropic.claude-3-5-sonnet-20240620-v1:0"

llm = AWSBedrockLLm(model_id=sonnet35)

print(llm.model)

client=<botocore.client.BedrockRuntime object at 0x7f4f14fcce00> model_id='anthropic.claude-3-5-sonnet-20240620-v1:0' provider='anthropic' supports_tool_choice_values=('auto', 'any', 'tool')


In [25]:
from prompt.prompt import SQL_PROMPT

SQL_PROMPT.with_inputs({'docs': 'hi doc', 'query': 'hi query'})

'You are a SQL expert, generate an executable postgres sql \nbased on the retireved docs and user question.\n Context docs: hi doc \n User question: hi query'

In [27]:
import sys
print(sys.executable)
print(sys.path)

/opt/conda/bin/python
['/opt/conda/lib/python312.zip', '/opt/conda/lib/python3.12', '/opt/conda/lib/python3.12/lib-dynload', '', '/opt/conda/lib/python3.12/site-packages', '/home/sagemaker-user/ChatBot/src']


In [28]:
import sys
import os

print("Python executable:", sys.executable)
print("sys.path:", sys.path)


Python executable: /opt/conda/bin/python
sys.path: ['/opt/conda/lib/python312.zip', '/opt/conda/lib/python3.12', '/opt/conda/lib/python3.12/lib-dynload', '', '/opt/conda/lib/python3.12/site-packages', '/home/sagemaker-user/ChatBot/src']


In [2]:
python -m ipykernel install --user --name=chatbot_venv --display-name "Python (.venv)"


SyntaxError: invalid syntax (1974878424.py, line 1)

In [1]:
from nodes.nodes import SQLNode
from state.state import State
from llm.llm import AWSBedrockLLm
from vectorstore.vectorstore import VectorStore
state = State()
llm = AWSBedrockLLm()
n1 = SQLNode('n1')

#n1.run({'docs': 'hi doc', 'query': 'hi query'}, state, llm)

In [3]:
print(n1.name)

n1


In [7]:
from prompt.prompt import SQL_PROMPT

print(SQL_PROMPT.with_inputs({'doc': "", 'query': ""}))

KeyError: 'docs'

In [12]:
SQL_PROMPT.template

PromptTemplate(input_variables=['docs', 'query'], input_types={}, partial_variables={}, template='You are a SQL expert, generate an executable postgres sql \nbased on the retireved docs and user question.\n Context docs: {docs} \n User question: {query}')

In [16]:
SQL_PROMPT.template.format_prompt(**{"docs": "", "query": ""})

StringPromptValue(text='You are a SQL expert, generate an executable postgres sql \nbased on the retireved docs and user question.\n Context docs:  \n User question: ')

In [ ]:
llm.invoke("HI")

AIMessage(content='Hello! How can I assist you today? Feel free to ask any questions or let me know if you need help with anything.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'fc0d059c-4614-4903-bcdc-292e4219fcd4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 26 Dec 2025 17:53:48 GMT', 'content-type': 'application/json', 'content-length': '314', 'connection': 'keep-alive', 'x-amzn-requestid': 'fc0d059c-4614-4903-bcdc-292e4219fcd4'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [1086]}, 'model_name': 'anthropic.claude-3-5-sonnet-20240620-v1:0'}, id='run--0fbddb5d-6551-400d-ac5d-c16e231f0f8f-0', usage_metadata={'input_tokens': 9, 'output_tokens': 29, 'total_tokens': 38, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

In [14]:
from langchain_core.prompts import PromptTemplate

# Define the prompt template
template = PromptTemplate(
    input_variables=['docs', 'query'],
    template=(
        "You are a SQL expert, generate an executable postgres sql \n"
        "based on the retrieved docs and user question.\n"
        "Context docs: {docs} \n"
        "User question: {query}"
    )
)

# Data to fill the template
data = {
    "docs": "Table users(id, name, age), Table orders(order_id, user_id, amount)",
    "query": "Get the total amount spent by each user."
}

# Fill the template
prompt_text = template.format(**data)
print(prompt_text)


You are a SQL expert, generate an executable postgres sql 
based on the retrieved docs and user question.
Context docs: Table users(id, name, age), Table orders(order_id, user_id, amount) 
User question: Get the total amount spent by each user.


In [ ]:
from documents.schema import all_docs


In [ ]:
from vectorstore.vectorstore import VectorStore


In [ ]:
vecstore = VectorStore(docs=all_docs)

In [ ]:
from prompt.prompt import *



In [ ]:
template = """ You are a SQL expert, use the following retrieved docs to generate sql to answer the user question. Only reply with a executaible postgres sql
Docs: 
{docs}

User Query: 
{query}
"""
ragprompt = CustomPrompt(template=template)

In [ ]:
userquery = "Count number of clients per state"

In [ ]:
retrieved_docs  = vecstore.retrieve(userquery)

In [ ]:
retrieved_docs

[Document(id='5a0831c4-5d2a-4193-b09b-a3a06425c33d', metadata={'type': 'metric', 'name': 'Client Census'}, page_content='  \nName: Client Census\nDescription: count of all the client for a given week from sunday to saturday\nSQL: SELECT COUNT(DISTINCT client_key) FROM dw_prod_report.vw_hours_and_client_census_weekly WHERE period_begin_date = (SELECT CURRENT_DATE - (EXTRACT(DOW FROM CURRENT_DATE)::integer + 7) AS last_full_week_sunday) AND CLIENT_SERVED_FLAG = TRUE AND client_key IS NOT NULL\n'),
 Document(id='20a536ac-9bf4-479b-a1d3-2f6efd34f75f', metadata={'type': 'metric_comparison', 'name': 'Client Census Week-over-Week Change', 'optimized': True, 'topic': 'census', 'granularity': 'weekly', 'dimension': 'state'}, page_content='\nName: Client Census Week-over-Week Comparison\nDescription: Count of distinct clients served per state for the last two full weeks (Sunday to Saturday), and the change in client count between those two weeks.\nSQL: WITH census_counts AS (\n  SELECT \n    sta

In [ ]:
from util import *

In [ ]:
formatted_docs = format_docs(retrieved_docs)
formatted_docs

"  \nName: Client Census\nDescription: count of all the client for a given week from sunday to saturday\nSQL: SELECT COUNT(DISTINCT client_key) FROM dw_prod_report.vw_hours_and_client_census_weekly WHERE period_begin_date = (SELECT CURRENT_DATE - (EXTRACT(DOW FROM CURRENT_DATE)::integer + 7) AS last_full_week_sunday) AND CLIENT_SERVED_FLAG = TRUE AND client_key IS NOT NULL\n\n\n\nName: Client Census Week-over-Week Comparison\nDescription: Count of distinct clients served per state for the last two full weeks (Sunday to Saturday), and the change in client count between those two weeks.\nSQL: WITH census_counts AS (\n  SELECT \n    state,\n    period_begin_date,\n    COUNT(DISTINCT client_key) AS client_count\n  FROM dw_prod_report.vw_hours_and_client_census_weekly_partitioned\n  WHERE CLIENT_SERVED_FLAG = TRUE\n    AND client_key IS NOT NULL\n    AND state IS NOT NULL\n    AND period_begin_date IN (\n      CURRENT_DATE - (EXTRACT(DOW FROM CURRENT_DATE)::integer + 7),\n      CURRENT_DATE

In [ ]:
from util import SQLQuery

In [ ]:
llm_structure = AWSBedrockLLm(model_id=sonnet35, structured_output=SQLQuery)

In [ ]:
ragresponse_structure = llm_structure.invoke(ragprompt.with_inputs({'docs': formatted_docs, 'query': userquery}))


In [ ]:
ragresponse = llm.invoke(ragprompt.with_inputs({'docs': formatted_docs, 'query': userquery}))


In [ ]:
ragresponse_structure

SQLQuery(question='Count the number of clients per state', sql='SELECT \n    state, \n    COUNT(DISTINCT client_key) AS client_count\nFROM \n    dw_prod_report.vw_hours_and_client_census_monthly\nWHERE \n    CLIENT_SERVED_FLAG = TRUE\n    AND client_key IS NOT NULL\n    AND state IS NOT NULL\nGROUP BY \n    state\nORDER BY \n    client_count DESC')

In [ ]:
ragresponse.content

"Here's the SQL query to count the number of clients per state:\n\n```sql\nSELECT \n    state, \n    COUNT(DISTINCT client_key) AS client_count\nFROM \n    dw_prod_report.vw_hours_and_client_census_monthly\nWHERE \n    CLIENT_SERVED_FLAG = TRUE \n    AND client_key IS NOT NULL \n    AND state IS NOT NULL\nGROUP BY \n    state\nORDER BY \n    client_count DESC;\n```\n\nThis query:\n1. Selects from the monthly client census view\n2. Counts distinct clients per state\n3. Filters for active clients (CLIENT_SERVED_FLAG = TRUE)\n4. Excludes null client keys and states\n5. Groups the results by state\n6. Orders the results by client count in descending order"

In [ ]:
ragresponse

AIMessage(content="Here's the SQL query to count the number of clients per state:\n\n```sql\nSELECT \n    state, \n    COUNT(DISTINCT client_key) AS client_count\nFROM \n    dw_prod_report.vw_hours_and_client_census_monthly\nWHERE \n    CLIENT_SERVED_FLAG = TRUE \n    AND client_key IS NOT NULL \n    AND state IS NOT NULL\nGROUP BY \n    state\nORDER BY \n    client_count DESC;\n```\n\nThis query:\n1. Selects from the monthly client census view\n2. Counts distinct clients per state\n3. Filters for active clients (CLIENT_SERVED_FLAG = TRUE)\n4. Excludes null client keys and states\n5. Groups the results by state\n6. Orders the results by client count in descending order", additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'bbc593b2-8b35-40c0-8758-98ee21cd31c1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 26 Dec 2025 17:57:28 GMT', 'content-type': 'application/json', 'content-length': '866', 'connection': 'keep-alive', 'x-amzn-requestid': 'bbc593b2-8b35-4

In [ ]:
ragresponse_structure.sql

'SELECT \n    state, \n    COUNT(DISTINCT client_key) AS client_count\nFROM \n    dw_prod_report.vw_hours_and_client_census_monthly\nWHERE \n    CLIENT_SERVED_FLAG = TRUE\n    AND client_key IS NOT NULL\n    AND state IS NOT NULL\nGROUP BY \n    state\nORDER BY \n    client_count DESC'

In [ ]:
llm_structure.invoke()